In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    dtype = torch.half,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.18 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [3]:
from datasets import load_dataset, concatenate_datasets
import pandas as pd

In [4]:
function_df = pd.read_csv("out.csv")
alpaca_df = pd.read_csv("out_alpaca.csv")

In [5]:
en_dataset = load_dataset('json', data_files={"Salesforce/xlam-function-calling-60k"}, split = "train")
en_dataset = en_dataset.remove_columns(["id"])

In [6]:
en_dataset["query"][0]

'Where can I find live giveaways for beta access and games?'

In [7]:
NUM_SAMPLES_TOTAL = 10000 #tr_dataset.num_rows
NUM_SAMPLES_TRAIN = int(NUM_SAMPLES_TOTAL * 0.9)
NUM_SAMPLES_INSTRUCT = int(NUM_SAMPLES_TRAIN * 0.3)

In [8]:
function_indices = function_df["2"].nlargest(NUM_SAMPLES_TOTAL).index
insruction_indices = alpaca_df["2"].nlargest(NUM_SAMPLES_INSTRUCT).index

In [9]:
function_calling_dataset = en_dataset.select(function_indices)
function_calling_dataset = function_calling_dataset.shuffle(seed=42)
function_calling_train = function_calling_dataset.select(range(NUM_SAMPLES_TRAIN))
function_calling_validation = function_calling_dataset.select(range(NUM_SAMPLES_TRAIN, NUM_SAMPLES_TOTAL)) #validation is only function calling

In [10]:
from unsloth.chat_templates import get_chat_template
import json
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)


def formatting_prompts_func(examples):
    query = examples["query"]
    answer = examples["answers"]
    convos = [{"role":"user","content":query},{"role":"assistant","content":answer}]
    tool_object = json.loads(examples["tools"])
    texts = tokenizer.apply_chat_template(convos,tools=tool_object,tokenize = False, add_generation_prompt = False)
    texts = texts.replace('"parameters": d','"arguments": d') # original tool use function uses parameters our dataset uses arguments

    return { "text" : texts, }
pass

function_calling_train= function_calling_train.map(formatting_prompts_func, batched = False,)
function_calling_validation = function_calling_validation.map(formatting_prompts_func, batched = False,)

In [11]:
function_calling_train[1]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nEnvironment: ipython\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.\n\nRespond in the format {"name": function name, "arguments": dictionary of argument name and its value}.Do not use variables.\n\n{\n    "name": "get_wisdom_quotes_by_topic",\n    "description": "Fetches wisdom quotes related to a specific topic using the RapidAPI service.",\n    "parameters": {\n        "topic": {\n            "description": "The topic to retrieve wisdom quotes about.",\n            "type": "str",\n            "default": "Anger"\n        }\n    }\n}\n\n{\n    "name": "get_types",\n    "description": "Fetches available types for a specified period from the horoscopes API.",\n    "parameters": {\n        "period": {\n            

In [12]:
print(function_calling_train[1]["text"])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.

Respond in the format {"name": function name, "arguments": dictionary of argument name and its value}.Do not use variables.

{
    "name": "get_wisdom_quotes_by_topic",
    "description": "Fetches wisdom quotes related to a specific topic using the RapidAPI service.",
    "parameters": {
        "topic": {
            "description": "The topic to retrieve wisdom quotes about.",
            "type": "str",
            "default": "Anger"
        }
    }
}

{
    "name": "get_types",
    "description": "Fetches available types for a specified period from the horoscopes API.",
    "parameters": {
        "period": {
            "description": "The time perio

In [13]:
def formatting_instruction_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    if inputs:
        convos = [{"role":"user","content":instructions + " " + inputs},{"role":"assistant","content":outputs}]
    else:
        convos = [{"role":"user","content":instructions},{"role":"assistant","content":outputs}]

    texts = tokenizer.apply_chat_template(convos,tokenize = False, add_generation_prompt = False)
    return { "text" : texts, }
pass


dataset_instruction = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset_instruction = dataset_instruction.map(formatting_instruction_prompts_func, batched = False,)

In [14]:
dataset_instruction = dataset_instruction.select(insruction_indices)
dataset_instruction = dataset_instruction.shuffle(seed=42)

In [15]:
NUM_SAMPLES_INSTRUCT_TRAIN = int(NUM_SAMPLES_INSTRUCT * 0.9)

instruction_train = dataset_instruction.select(range(NUM_SAMPLES_INSTRUCT_TRAIN))
instruction_validation = dataset_instruction.select(range(NUM_SAMPLES_INSTRUCT_TRAIN, NUM_SAMPLES_INSTRUCT))

In [16]:
train_dataset = concatenate_datasets([function_calling_train, instruction_train])

In [17]:
validation_set = concatenate_datasets([function_calling_validation, instruction_validation])

In [18]:
train_dataset = train_dataset.shuffle(seed=42)
validation_set = validation_set.shuffle(seed=42)

In [19]:
validation_set["text"][1]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nEnvironment: ipython\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.\n\nRespond in the format {"name": function name, "arguments": dictionary of argument name and its value}.Do not use variables.\n\n{\n    "name": "can_attend_all_meetings",\n    "description": "Determines if a person can attend all meetings given a list of meeting time intervals.",\n    "parameters": {\n        "intervals": {\n            "description": "A list of meeting time intervals, where each interval is represented as [start_time, end_time].",\n            "type": "List[List[int]]"\n        }\n    }\n}\n\n{\n    "name": "fibonacci_numbers",\n    "description": "Generates the first n Fibonacci numbers.",\n    "parameters": {\n        "n": {

In [20]:
import torch
torch.cuda.empty_cache()

In [21]:
import wandb
import os
wandb.init(mode="disabled")
os.environ["WANDB_DISABLED"] = "true"
#os.environ["UNSLOTH_RETURN_LOGITS"] = "1"

In [22]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
5.748 GB of memory reserved.


In [25]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from transformers import TrainingArguments, DataCollatorForSeq2Seq

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = validation_set,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 32,
        per_device_eval_batch_size = 8,
        gradient_accumulation_steps = 1,
        evaluation_strategy="steps",
        batch_eval_metrics=True,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        learning_rate = 1e-4,
        fp16 = True,#not is_bfloat16_supported(),
        bf16 = False,#is_bfloat16_supported(),
        logging_steps = 100,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "train_output",
        report_to="none"
    ),
)

from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [26]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 11,430 | Num Epochs = 1 | Total steps = 358
O^O/ \_/ \    Batch size per device = 32 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (32 x 1 x 1) = 32
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Step,Training Loss,Validation Loss
100,0.214000,0.159512
200,0.158300,0.155043
300,0.156600,0.153195


Unsloth: Will smartly offload gradients to save VRAM!


In [27]:
path = "model_weights"

model.save_pretrained(path) # Local saving
tokenizer.save_pretrained(path)

('/content/drive/MyDrive/Private code/Function calling LLM - EN/model_weights/tokenizer_config.json',
 '/content/drive/MyDrive/Private code/Function calling LLM - EN/model_weights/special_tokens_map.json',
 '/content/drive/MyDrive/Private code/Function calling LLM - EN/model_weights/tokenizer.json')

In [28]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "model_weights",
    max_seq_length = max_seq_length,
    dtype = torch.half,
)

==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [33]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [35]:
instruction = "What is the integral of cos(x)"
convos = [{"role":"user","content":instruction}]
texts = tokenizer.apply_chat_template(convos,tokenize = False, add_generation_prompt = True)

inputs = tokenizer(
[    texts
], return_tensors = "pt").to("cuda")


from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = False)
outputs = model.generate(**inputs, max_new_tokens = 4096, streamer = text_streamer)

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the integral of cos(x)<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The integral of cos(x) is sin(x).<|eot_id|>


In [50]:
function_calling_validation

Dataset({
    features: ['query', 'answers', 'tools', 'text'],
    num_rows: 1000
})

In [51]:
index = 0
query = function_calling_validation["query"][index]
convos = [{"role":"user","content":query}]
tool_object = json.loads(function_calling_validation["tools"][index])
texts = tokenizer.apply_chat_template(convos,tools=tool_object,tokenize = False, add_generation_prompt = False)
texts = texts.replace('"parameters": d','"arguments": d') # original tool use function uses parameters our dataset uses arguments

In [54]:
convos = [{"role":"user","content":texts}]
texts = tokenizer.apply_chat_template(convos,tokenize = False, add_generation_prompt = True)

inputs = tokenizer(
[    texts
], return_tensors = "pt").to("cuda")


from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
outputs = model.generate(**inputs, max_new_tokens = 4096, streamer = text_streamer)

[{"name": "calculate_angle", "arguments": {"hour": 11, "minute": 59}}]<|eot_id|>


In [56]:
print(function_calling_validation[index]["answers"])

[{"name": "calculate_angle", "arguments": {"hour": 11, "minute": 59}}]


In [57]:
index = 1
query = function_calling_validation["query"][index]
convos = [{"role":"user","content":query}]
tool_object = json.loads(function_calling_validation["tools"][index])
texts = tokenizer.apply_chat_template(convos,tools=tool_object,tokenize = False, add_generation_prompt = False)
texts = texts.replace('"parameters": d','"arguments": d') # original tool use function uses parameters our dataset uses arguments
convos = [{"role":"user","content":texts}]
texts = tokenizer.apply_chat_template(convos,tokenize = False, add_generation_prompt = True)

inputs = tokenizer(
[    texts
], return_tensors = "pt").to("cuda")


from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
outputs = model.generate(**inputs, max_new_tokens = 4096, streamer = text_streamer)

[{"name": "structural_analysis", "arguments": {"building_id": "B2", "floor_numbers": [4, 5, 6], "analysis_mode": "dynamic"}}]<|eot_id|>


In [58]:
print(function_calling_validation[index]["answers"])

[{"name": "structural_analysis", "arguments": {"building_id": "B2", "floor_numbers": [4, 5, 6], "analysis_mode": "dynamic"}}]
